In [ ]:
# Load model
model = joblib.load('random_forest_model.pkl')

# Load scalers
X_scaler = joblib.load('X_scaler.pkl')
Y_scaler = joblib.load('Y_scaler.pkl')
with open('ski_areas.geojson', 'r') as f:
    ski_areas = json.load(f)
crystal_mountain_area = [site for site in ski_areas['features'] if site['properties']['name'] == 'Crystal Mountain'][0]
crystal_polygon = shape(crystal_mountain_area['geometry'])
crystal_polygon

In [ ]:
# Load data
results_df = pd.read_parquet('results.parquet')
results_df = results_df.dropna(subset=['fsca'])
results_df['datetime'] = pd.to_datetime(results_df['time'])
results_df['day'] = results_df['datetime'].dt.day
results_df['month'] = results_df['datetime'].dt.month
results_df['year'] = results_df['datetime'].dt.year
results_df.loc[~results_df['fsca'].between(0, 1000), 'fsca'] = None
results_df.head()

input_columns = ["red", "green", "blue", "coastal", "nir08", "swir16", "swir22", "fsca", "latitude", "longitude", "month"]
x_input_data = results_df[input_columns]
x_scaled = X_scaler.transform(x_input_data)

y_pred = model.predict(x_scaled)
y_pred
y_scaled = Y_scaler.inverse_transform([[pred] for pred in y_pred])
y_scaled
results_df['snow_depth_prediction'] = y_scaled
results_df.shape
# reduce overlapping pixels: group by day, lat, lon and pick the highest value for that pixel
max_by_day = results_df.groupby(['year', 'month', 'day', 'latitude', 'longitude']).agg({'snow_depth_prediction': 'max'}).reset_index(level=[3,4])
# make sure that there are pixels that cover the entire polygon for each day
def check_daily_coverage(df, target_polygon):
    # Create points from your DataFrame
    points = [Point(lon, lat) for lon, lat in zip(df['longitude'], df['latitude'])]

    # Check if all points are within the polygon
    points_in_polygon = [point.within(target_polygon) for point in points]
    coverage_percentage = sum(points_in_polygon) / len(points_in_polygon) * 100
    return coverage_percentage

for group in max_by_day:
    coverage_percentage = check_daily_coverage(group, crystal_polygon)
    if coverage_percentage < 100:
        print(f"Day {group.day} has {coverage_percentage}% coverage")
        # remove from group

avg_by_month = max_by_day.groupby(['year', 'month']).agg({'snow_depth_prediction': 'mean'})

# what we have a matrix of observations, identified by datetime and a snow depth prediction for each pixel.
# We can multiply those by the size of each pixel to get the area of snow.
# Then we can sum those up to get the total area of snow for each year.
area_of_each_pixel = 30 * 30
avg_by_month['snow_volume_prediction'] = avg_by_month['snow_depth_prediction'] * area_of_each_pixel
avg_by_month['snow_volume_prediction']
# aggregate over the season (November through April)
avg_by_month = avg_by_month[avg_by_month['month'].isin(range(11, 5))]
sum_over_season = avg_by_month.groupby('year').agg({'snow_volume_prediction': 'sum'})
sum_over_season.plot(kind='line', x='year', y='snow_volume_prediction')
plt.show()
